<h1 align="center">  Chatbot -modelisation. </h1>  
</br>


In [1]:
import json
import string
import random
import nltk
#nltk.download('wordnet')
#nltk.download('omw-1.4')
import numpy as np
from nltk.stem import WordNetLemmatizer# It has the ability to lemmatize.
import tensorflow as tensorF # A multidimensional array of elements is represented by this symbol.
from tensorflow.keras import Sequential # Sequential groups a linear stack of layers into a tf.keras.Model
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
import speech_recognition as sr

import warnings
warnings.filterwarnings('ignore')

## Step one: Importing libraries


## Step two: Creating a JSON file

In [2]:
def get_data():
    with open("data.json", 'r', encoding='utf-8') as f:
        data = json.load(f)
    return data

## Step three: Processing data


In [3]:
def data_processing(data):
    lm = WordNetLemmatizer() #for getting words
    # lists
    ourClasses = []
    newWords = []
    documentX = []
    documentY = []
    # Each intent is tokenized into words and the patterns and their associated tags are added to their respective lists.
    for intent in data["Intents"]:
        for pattern in intent["patterns"]:
            ournewTkns = nltk.word_tokenize(pattern)# tokenize the patterns
            newWords.extend(ournewTkns)# extends the tokens
            documentX.append(pattern)
            documentY.append(intent["tag"])


        if intent["tag"] not in ourClasses:# add unexisting tags to their respective classes
            ourClasses.append(intent["tag"])

    newWords = [lm.lemmatize(word) for word in newWords if word not in string.punctuation] # set words to lowercase if not in punctuation
    newWords = sorted(set(newWords))# sorting words
    ourClasses = sorted(set(ourClasses))# sorting classes
    return (newWords,ourClasses,documentX,documentY)

In [4]:
get_data()

{'Intents': [{'tag': 'age',
   'patterns': ['كم عمرك ؟ ', 'قداش عمرك؟'],
   'responses': ['لقد تمت برمجتي من قبل ملاك يعيش و محمد امين اللحياني يوم 1 ماي 2023 كجزء من مشروع اكاديمي.اتم اليوم اذا يومي السادس ']},
  {'tag': 'greeting',
   'patterns': ['صباح النور',
    'مرحبا',
    'سلام علیکم',
    'أهلاً',
    'صباح الخير',
    'سلام'],
   'responses': ['عسلامة، أنا الدكتور حكيم. كيفلش نجم نعاونك؟',
    'سلام، انا الدكتور حكيم يمكنك طرح اي استشارة طبية ',
    ' مرحبا بيك، ماهو سؤالك اليوم ؟',
    'صباح الخير،الدكتور حكيم حاضر لاي سؤال ']},
  {'tag': 'goodbye',
   'patterns': ['شكرا', 'مع السلامة'],
   'responses': ['سررت بمساعتك',
    'اتمنى ان تكون قد استفدت من الاجابة، بااي  ']},
  {'tag': 'name',
   'patterns': ['ماهو اسمك؟', 'من انت؟', 'ماهي وظيفتك؟'],
   'responses': ['أنا الدكتور حكيم، روبوت دردشة. وظيفتي هي تقديم نصائح طبية لك']},
  {'tag': 'الامتحان',
   'patterns': ['انا قلقة من الإمتحان',
    'من فضلكم كيف أتخلص من التوتر والخوف في الامتحان',
    'كيف اتغلب على توتر الامتحان 

## Step four: Designing a neural network model


In [5]:
newWords,ourClasses,documentX,documentY= data_processing(get_data())
lm = WordNetLemmatizer() #for getting words

trainingData = [] # training list array
outEmpty = [0] * len(ourClasses)
# bow model
for idx, doc in enumerate(documentX):
    bagOfwords = []
    text = lm.lemmatize(doc)
    for word in newWords:
        bagOfwords.append(1) if word in text else bagOfwords.append(0)

    outputRow = list(outEmpty)
    outputRow[ourClasses.index(documentY[idx])] = 1
    trainingData.append([bagOfwords, outputRow])

random.shuffle(trainingData)
trainingData = np.array(trainingData, dtype=object)# coverting our data into an array afterv shuffling

x = np.array(list(trainingData[:, 0]))# first trainig phase
y = np.array(list(trainingData[:, 1]))# second training phase

In [6]:
iShape = (len(x[0]),)
oShape = len(y[0])
# parameter definition
ourNewModel = Sequential()
# In the case of a simple stack of layers, a Sequential model is appropriate

# Dense function adds an output layer
ourNewModel.add(Dense(128, input_shape=iShape, activation="relu"))
# The activation function in a neural network is in charge of converting the node's summed weighted input into activation of the node or output for the input in question
ourNewModel.add(Dropout(0.5))
# Dropout is used to enhance visual perception of input neurons
ourNewModel.add(Dense(64, activation="relu"))
ourNewModel.add(Dropout(0.3))
ourNewModel.add(Dense(oShape, activation = "softmax"))

# Below line improves the numerical stability and pushes the computation of the probability distribution into the categorical crossentropy loss function.
ourNewModel.compile(loss='categorical_crossentropy', optimizer=tensorF.keras.optimizers.Adam(lr=0.01), metrics=["accuracy"])
# Output the model in summary
print(ourNewModel.summary())
# Whilst training your Nural Network, you have the option of making the output verbose or simple.
ourNewModel.fit(x, y, epochs=200, verbose=1)

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 128)               34688     
                                                                 
 dropout (Dropout)           (None, 128)               0         
                                                                 
 dense_1 (Dense)             (None, 64)                8256      
                                                                 
 dropout_1 (Dropout)         (None, 64)                0         
                                                                 
 dense_2 (Dense)             (None, 15)                975       
                                                                 
Total params: 43,919
Trainable params: 43,919
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/200
2/2 [==============================

Epoch 156/200
2/2 [==============================] - 0s 13ms/step - loss: 0.0870 - accuracy: 0.9815
Epoch 157/200
2/2 [==============================] - 0s 8ms/step - loss: 0.0983 - accuracy: 0.9815
Epoch 158/200
2/2 [==============================] - 0s 9ms/step - loss: 0.1204 - accuracy: 0.9630
Epoch 159/200
2/2 [==============================] - 0s 8ms/step - loss: 0.0854 - accuracy: 0.9815
Epoch 160/200
2/2 [==============================] - 0s 6ms/step - loss: 0.0687 - accuracy: 1.0000
Epoch 161/200
2/2 [==============================] - 0s 8ms/step - loss: 0.1062 - accuracy: 0.9815
Epoch 162/200
2/2 [==============================] - 0s 9ms/step - loss: 0.0938 - accuracy: 0.9815
Epoch 163/200
2/2 [==============================] - 0s 7ms/step - loss: 0.0957 - accuracy: 0.9630
Epoch 164/200
2/2 [==============================] - 0s 7ms/step - loss: 0.1520 - accuracy: 0.9444
Epoch 165/200
2/2 [==============================] - 0s 8ms/step - loss: 0.0725 - accuracy: 0.9815
Epoch 166

In [7]:
model_json = ourNewModel.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)
ourNewModel.save_weights('model.h5') 

## Step five: Building useful features


In [8]:
def ourText(text):
    lm = WordNetLemmatizer()
    newtkns = nltk.word_tokenize(text)
    newtkns = [lm.lemmatize(word) for word in newtkns]
    return newtkns

def wordBag(text, vocab):
    newtkns = ourText(text)
    bagOwords = [0] * len(vocab)
    for w in newtkns:
        for idx, word in enumerate(vocab):
            if word == w:
                bagOwords[idx] = 1
    return np.array(bagOwords)

def Pclass(text, vocab, labels,model):
    bagOwords = wordBag(text, vocab)
    ourResult = model.predict(np.array([bagOwords]))[0]
    newThresh = 0.2
    yp = [[idx, res] for idx, res in enumerate(ourResult) if res > newThresh]

    yp.sort(key=lambda x: x[1], reverse=True)
    newList = []
    for r in yp:
        newList.append(labels[r[0]])
    return newList

def getRes(firstlist, fJson):
    tag = firstlist[0]
    listOfIntents = fJson["Intents"]
    for i in listOfIntents:
        if i["tag"] == tag:
            ourResult = random.choice(i["responses"])
            break
    return ourResult

## Execution

In [9]:
def get_answer(text,model):
    data=get_data()
    newWords, ourClasses,_,__=data_processing(data)
    intents = Pclass(text, newWords, ourClasses,model)
    return getRes(intents, data)
            

In [10]:
get_answer("سلام",ourNewModel)

1/1 [==============================] - 0s 147ms/step


' مرحبا بيك، ماهو سؤالك اليوم ؟'